# point_defect_diffusion - Methodology and code

__Python imports__:

- [numpy](http://www.numpy.org/)
- [DataModelDict](https://github.com/usnistgov/DataModelDict)
- [atomman](https://github.com/usnistgov/atomman)
- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from pathlib import Path
import shutil
import random
import datetime
from copy import deepcopy
from typing import Optional, Union

# http://www.numpy.org/
import numpy as np 

# https://ipython.org/
from IPython.display import display, Markdown

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc
from atomman.tools import filltemplate

# https://github.com/usnistgov/iprPy
import iprPy
from iprPy.tools import read_calc_file

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

Notebook last executed on 2024-05-02 using iprPy version 0.11.7


## 1. Load calculation and view description

### 1.1. Load the calculation

In [2]:
# Load the calculation being demoed
calculation = iprPy.load_calculation('point_defect_diffusion')

### 1.2. Display calculation description and theory

In [3]:
# Display main docs and theory
display(Markdown(calculation.maindoc))
display(Markdown(calculation.theorydoc))

# point_defect_diffusion calculation style

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

Description updated: 2019-07-26

## Introduction

The point_defect_diffusion calculation style estimates the diffusion rate of a point defect at a specified temperature.  A system is created with a single point defect, and subjected to a long time molecular dynamics simulation.  The mean square displacement for the defect is computed, and used to estimate the diffusion constant.

### Version notes

- 2022-03-11: Notebook updated to reflect version 0.11.

### Additional dependencies

### Disclaimers

- [NIST disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)
- The calculation estimates the defect's diffusion by computing the mean square displacement of all atoms in the system.  This is useful for estimating rates associated with vacancies and self-interstitials as the process is not confined to a single atom's motion.  However, this makes the calculation ill-suited to measuring diffusion of substitutional impurities as it does not individually track each atom's position throughout.


## Method and Theory

First, a defect system is constructed by adding a single point defect (or defect cluster) to an initially bulk system using the atomman.defect.point() function.

A LAMMPS simulation is then performed on the defect system.  The simulation consists of two separate runs

1. NVT equilibrium run: The system is allowed to equilibrate at the given temperature using nvt integration.

2. NVE measurement run: The system is then evolved using nve integration, and the total mean square displacement of all atoms is measured as a function of time.

Between the two runs, the atomic velocities are scaled such that the average temperature of the nve run is closer to the target temperature.

The mean square displacement of the defect, $\left< \Delta r_{ptd}^2 \right>$ is then estimated using the mean square displacement of the atoms $\left< \Delta r_{i}^2 \right>$.  Under the assumption that all diffusion is associated with the single point defect, the defect's mean square displacement can be taken as the summed square displacement of the atoms

$$ \left< \Delta r_{ptd}^2 \right> \approx \sum_i^N \Delta r_{i}^2 = N \left< \Delta r_{i}^2 \right>, $$

where $N$ is the number of atoms in the system.  The diffusion constant is then estimated by linearly fitting the change in mean square displacement with time

$$ \left< \Delta r_{ptd}^2 \right> = 2 d D_{ptd} \Delta t, $$

where d is the number of dimensions included.


## 2. Define calculation functions and generate files

This section defines the calculation functions and associated resource files exactly as they exist inside the iprPy package.  This allows for the code used to be directly visible and modifiable by anyone looking to see how it works.

### 2.1. pointdiffusion()

This is the primary function for the calculation.  The version of this function built in iprPy can be accessed by calling the calc() method of an object of the associated calculation class.

In [4]:
def pointdiffusion(lammps_command: str,
                   system: am.System,
                   potential: lmp.Potential,
                   point_kwargs: Union[list, dict],
                   mpi_command: Optional[str] = None,
                   temperature: float = 300.0,
                   runsteps: int = 200000,
                   thermosteps: Optional[int] = None,
                   dumpsteps: int = 0,
                   equilsteps: int = 20000,
                   randomseed: Optional[int] = None) -> dict:
                   
    """
    Evaluates the diffusion rate of a point defect at a given temperature. This
    method will run two simulations: an NVT run at the specified temperature to 
    equilibrate the system, then an NVE run to measure the defect's diffusion 
    rate. The diffusion rate is evaluated using the mean squared displacement of
    all atoms in the system, and using the assumption that diffusion is only due
    to the added defect(s).
    
    Parameters
    ----------
    lammps_command :str
        Command for running LAMMPS.
    system : atomman.System
        The system to perform the calculation on.
    potential : atomman.lammps.Potential
        The LAMMPS implemented potential to use.
    point_kwargs : dict or list of dict
        One or more dictionaries containing the keyword arguments for
        the atomman.defect.point() function to generate specific point
        defect configuration(s).
    mpi_command : str, optional
        The MPI command for running LAMMPS in parallel.  If not given, LAMMPS
        will run serially.
    temperature : float, optional
        The temperature to run at (default is 300.0).
    runsteps : int, optional
        The number of integration steps to perform (default is 200000).
    thermosteps : int, optional
        Thermo values will be reported every this many steps (default is
        100).
    dumpsteps : int or None, optional
        Dump files will be saved every this many steps (default is 0,
        which does not output dump files).
    equilsteps : int, optional
        The number of timesteps at the beginning of the simulation to
        exclude when computing average values (default is 20000).
    randomseed : int or None, optional
        Random number seed used by LAMMPS in creating velocities and with
        the Langevin thermostat.  (Default is None which will select a
        random int between 1 and 900000000.)
    
    Returns
    -------
    dict
        Dictionary of results consisting of keys:
        
        - **'natoms'** (*int*) - The number of atoms in the system.
        - **'temp'** (*float*) - The mean measured temperature.
        - **'pxx'** (*float*) - The mean measured normal xx pressure.
        - **'pyy'** (*float*) - The mean measured normal yy pressure.
        - **'pzz'** (*float*) - The mean measured normal zz pressure.
        - **'Epot'** (*numpy.array*) - The mean measured total potential 
          energy.
        - **'temp_std'** (*float*) - The standard deviation in the measured
          temperature values.
        - **'pxx_std'** (*float*) - The standard deviation in the measured
          normal xx pressure values.
        - **'pyy_std'** (*float*) - The standard deviation in the measured
          normal yy pressure values.
        - **'pzz_std'** (*float*) - The standard deviation in the measured
          normal zz pressure values.
        - **'Epot_std'** (*float*) - The standard deviation in the measured
          total potential energy values.
        - **'dx'** (*float*) - The computed diffusion constant along the 
          x-direction.
        - **'dy'** (*float*) - The computed diffusion constant along the 
          y-direction.
        - **'dz'** (*float*) - The computed diffusion constant along the 
          y-direction.
        - **'d'** (*float*) - The total computed diffusion constant.
    """

    # Add defect(s) to the initially perfect system
    if not isinstance(point_kwargs, (list, tuple)):
        point_kwargs = [point_kwargs]
    for pkwargs in point_kwargs:
        #print(pkwargs)
        system = am.defect.point(system, **pkwargs)
    
    # Get lammps units
    lammps_units = lmp.style.unit(potential.units)
    
    #Get lammps version date
    lammps_date = lmp.checkversion(lammps_command)['date']
    
    # Check that temperature is greater than zero
    if temperature <= 0.0:
        raise ValueError('Temperature must be greater than zero')
    
    # Handle default values
    if thermosteps is None: 
        thermosteps = runsteps // 1000
        if thermosteps == 0:
            thermosteps = 1
    if dumpsteps is None:
        dumpsteps = runsteps
    if randomseed is None:
        randomseed = random.randint(1, 900000000)
    
    # Define lammps variables
    lammps_variables = {}
    system_info = system.dump('atom_data', f='initial.dat',
                              potential=potential)
    lammps_variables['atomman_system_pair_info'] = system_info
    lammps_variables['temperature'] = temperature
    lammps_variables['runsteps'] = runsteps
    lammps_variables['equilsteps'] = equilsteps
    lammps_variables['thermosteps'] = thermosteps
    lammps_variables['dumpsteps'] = dumpsteps
    lammps_variables['randomseed'] = randomseed
    lammps_variables['timestep'] = lmp.style.timestep(potential.units)
    
    # Set dump_info
    if dumpsteps == 0:
        lammps_variables['dump_info'] = ''
    else:
        lammps_variables['dump_info'] = '\n'.join([
            '',
            '# Define dump files',
            'dump dumpit all custom ${dumpsteps} *.dump id type x y z c_peatom',
            'dump_modify dumpit format <dump_modify_format>',
            '',
        ])
        
        # Set dump_modify_format based on lammps_date
        if lammps_date < datetime.date(2016, 8, 3):
            lammps_variables['dump_modify_format'] = '"%d %d %.13e %.13e %.13e %.13e"'
        else:
            lammps_variables['dump_modify_format'] = 'float %.13e'
    
    # Write lammps input script
    lammps_script = 'diffusion.in'
    template = read_calc_file('iprPy.calculation.point_defect_diffusion',
                              'diffusion.template')
    with open(lammps_script, 'w') as f:
        f.write(filltemplate(template, lammps_variables, '<', '>'))
    
    # Run lammps
    output = lmp.run(lammps_command, script_name=lammps_script,
                     mpi_command=mpi_command)
    
    # Extract LAMMPS thermo data.
    thermo = output.simulations[1]['thermo']
    temps = thermo.Temp.values
    pxxs = uc.set_in_units(thermo.Pxx.values, lammps_units['pressure'])
    pyys = uc.set_in_units(thermo.Pyy.values, lammps_units['pressure'])
    pzzs = uc.set_in_units(thermo.Pzz.values, lammps_units['pressure'])
    E_pots = uc.set_in_units(thermo.PotEng.values, lammps_units['energy'])
    E_totals = uc.set_in_units(thermo.TotEng.values, lammps_units['energy'])
    steps = thermo.Step.values
    
    # Read user-defined thermo data
    if output.lammps_date < datetime.date(2016, 8, 1):
        msd_x = uc.set_in_units(thermo['msd[1]'].values,
                                lammps_units['length']+'^2')
        msd_y = uc.set_in_units(thermo['msd[2]'].values,
                                lammps_units['length']+'^2')
        msd_z = uc.set_in_units(thermo['msd[3]'].values,
                                lammps_units['length']+'^2')
        msd = uc.set_in_units(thermo['msd[4]'].values,
                              lammps_units['length']+'^2')
    else:
        msd_x = uc.set_in_units(thermo['c_msd[1]'].values,
                                lammps_units['length']+'^2')
        msd_y = uc.set_in_units(thermo['c_msd[2]'].values,
                                lammps_units['length']+'^2')
        msd_z = uc.set_in_units(thermo['c_msd[3]'].values,
                                lammps_units['length']+'^2')
        msd = uc.set_in_units(thermo['c_msd[4]'].values,
                              lammps_units['length']+'^2')
        
    # Initialize results dict
    results = {}
    results['natoms'] = system.natoms
    results['nsamples'] = len(thermo)
    
    # Get mean and std for temperature, pressure, and energy
    results['temp'] = np.mean(temps)
    results['temp_std'] = np.std(temps)
    results['pxx'] = np.mean(pxxs)
    results['pxx_std'] = np.std(pxxs)
    results['pyy'] = np.mean(pyys)
    results['pyy_std'] = np.std(pyys)
    results['pzz'] = np.mean(pzzs)
    results['pzz_std'] = np.std(pzzs)
    results['E_pot'] = np.mean(E_pots)
    results['E_pot_std'] = np.std(E_pots)
    results['E_total'] = np.mean(E_totals)
    results['E_total_std'] = np.std(E_totals)
    
    # Convert steps to times
    times = steps * uc.set_in_units(lammps_variables['timestep'], lammps_units['time'])
    
    # Estimate diffusion rates
    # MSD_ptd = natoms * MSD_atoms (if one defect in system)
    # MSD = 2 * ndim * D * t  -->  D = MSD/t / (2 * ndim)
    mx = np.polyfit(times, system.natoms * msd_x, 1)[0]
    my = np.polyfit(times, system.natoms * msd_y, 1)[0]
    mz = np.polyfit(times, system.natoms * msd_z, 1)[0]
    m = np.polyfit(times, system.natoms * msd, 1)[0]

    results['msd_x_values'] = msd_x
    results['msd_y_values'] = msd_y
    results['msd_z_values'] = msd_z
    results['msd_values'] = msd
    results['dx'] = mx / 2
    results['dy'] = my / 2
    results['dz'] = mz / 2
    results['d'] = m / 6
    
    return results

### 2.2. diffusion.template file

In [5]:
with open('diffusion.template', 'w') as f:
    f.write("""# LAMMPS input script for dynamic msd computation

box tilt large

<atomman_system_pair_info>

# Assign simulation parameter values
variable temperature equal <temperature>
variable randomseed equal <randomseed>
variable thermosteps equal <thermosteps>
variable timestep equal <timestep>
variable equilsteps equal <equilsteps>
variable dumpsteps equal <dumpsteps>
variable runsteps equal <runsteps>
variable twotemp equal 2*${temperature}
variable damptemp equal 100*${timestep}

# Specify property computes
compute peatom all pe/atom
compute msd all msd com yes

# Define thermo data
thermo ${thermosteps}
thermo_style custom step temp pe ke etotal pxx pyy pzz c_msd[1] c_msd[2] c_msd[3] c_msd[4]
thermo_modify format float %.13e

# Specify timestep
timestep ${timestep}

# Create velocities and equilibrate system using nvt
velocity all create ${twotemp} ${randomseed}
fix 1 all nvt temp ${temperature} ${temperature} ${damptemp}
run ${equilsteps}
unfix 1
<dump_info>

# Scale velocities to wanted temperature and run nve
velocity all scale ${temperature}
reset_timestep 0
fix 2 all nve
run ${runsteps}""")

## 3. Specify input parameters

### 3.1. System-specific paths

- __lammps_command__ is the LAMMPS command to use (required).
- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [6]:
lammps_command = '/home/lmh1/LAMMPS/2022-06-23/src/lmp_mpi'
mpi_command = 'mpiexec -n 8'

# Optional: check that LAMMPS works and show its version 
print(f'LAMMPS version = {am.lammps.checkversion(lammps_command)["version"]}')

LAMMPS version = 23 Jun 2022


### 3.2. Interatomic potential

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for the calculation.  
- __potential__ is an atomman.lammps.Potential object (required).

In [7]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'

# Retrieve potential and parameter file(s) using atomman
potential = am.load_lammps_potential(id=potential_name, getfiles=True)

### 3.3. Initial unit cell system

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).  Here, this is generated using the load parameters and symbols.

In [8]:
# Create ucell by loading prototype record
ucell = am.load('crystal', potential=potential, family='A1--Cu--fcc')

print(ucell)

Multiple matching record retrieved from local
#  family               symbols  alat    Ecoh    method  standing
 1 A1--Cu--fcc          Ni        3.5200 -4.4500 dynamic good
 2 A1--Cu--fcc          Ni        7.3760  0.0119 dynamic good


Please select one: 1


avect =  [ 3.520,  0.000,  0.000]
bvect =  [ 0.000,  3.520,  0.000]
cvect =  [ 0.000,  0.000,  3.520]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Ni',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.000 |   1.760 |   1.760
      2 |       1 |   1.760 |   0.000 |   1.760
      3 |       1 |   1.760 |   1.760 |   0.000


### 3.4 Defect parameters

- __point_kwargs__ (required) is a dictionary or list of dictonaries containing parameters for generating the defect. Here, values are extracted from pointdefect_file. Allowed keywords are:

    - __ptd_type__ indicates which defect type to generate: 'v' for vacancy, 'i' for interstitial, 's' for substitutional, or 'db' for dumbbell.
    
    - __atype__ is the atom type to assign to the defect atom ('i', 's', 'db' ptd_types).
    
    - __pos__ specifies the position for adding the defect atom (all ptd_types).
    
    - __ptd_id__ specifies the id of an atom in the initial system where the defect is to be added. Alternative to using pos ('v', 's', 'db' ptd_types).
    
    - __db_vect__ gives the vector associated with the dumbbell interstitial to generate ('db' ptd_type).
    
    - __scale__ indicates if pos and db_vect are in absolute (False) or box-relative (True) coordinates. Default is False.
    
    - __atol__ is the absolute tolerance for position-based searching. Default is 1e-3 angstroms.

In [9]:
# Add a vacancy by deleting the atom at [0,0,0] - works for any crystal structure
point_kwargs = [{
    'ptd_type': 'v',
    'pos': np.array([0.0, 0.0, 0.0]),
    'scale': True,
}]

### 3.5. System modifications

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

- __system__ is an atomman.System to perform the scan on (required). 

In [10]:
sizemults = [10, 10, 10]

# Generate system by supersizing ucell
system = ucell.supersize(*sizemults)
print('# of atoms in system =', system.natoms)

# of atoms in system = 4000


### 3.6 Calculation-specific parameters

- __temperature__ temperature in Kelvin at which to run the MD integration scheme at.  Default value is '0'.

- __runsteps__ specifies how many timesteps to integrate the system.  Default value is 200000.

- __thermosteps__ specifies how often LAMMPS prints the system-wide thermo data.  Default value is runsteps/1000, or 1 if runsteps is less than 1000.
    
- __dumpsteps__ specifies how often LAMMPS saves the atomic configuration to a LAMMPS dump file.  Default value is runsteps, meaning only the first and last states are saved.
    
- __equilsteps__ specifies how many timesteps are ignored as equilibration time when computing the mean box parameters.  Default value is 10000.

- __randomseed__ provides a random number seed to generating the initial atomic velocities.  Default value gives a random number as the seed.

In [11]:
temperature = 1900
runsteps = 200000
thermosteps = 100
dumpsteps = runsteps
equilsteps = 20000
randomseed = None

## 4. Run calculation and view results

### 4.1. Run calculation

All primary calculation method functions take a series of inputs and return a dictionary of outputs.

In [12]:
results_dict = pointdiffusion(lammps_command, system, potential, point_kwargs,
                              mpi_command = mpi_command,
                              temperature = temperature,
                              runsteps = runsteps,
                              thermosteps = thermosteps,
                              dumpsteps = dumpsteps,
                              equilsteps = equilsteps,
                              randomseed = randomseed)
results_dict.keys()

dict_keys(['natoms', 'nsamples', 'temp', 'temp_std', 'pxx', 'pxx_std', 'pyy', 'pyy_std', 'pzz', 'pzz_std', 'E_pot', 'E_pot_std', 'E_total', 'E_total_std', 'msd_x_values', 'msd_y_values', 'msd_z_values', 'msd_values', 'dx', 'dy', 'dz', 'd'])

### 4.2. Report results

Values returned in the results_dict:

- **'natoms'** (*int*) - The number of atoms in the system.
- **'temp'** (*float*) - The mean measured temperature.
- **'pxx'** (*float*) - The mean measured normal xx pressure.
- **'pyy'** (*float*) - The mean measured normal yy pressure.
- **'pzz'** (*float*) - The mean measured normal zz pressure.
- **'Epot'** (*numpy.array*) - The mean measured total potential energy.
- **'temp_std'** (*float*) - The standard deviation in the measured temperature values.
- **'pxx_std'** (*float*) - The standard deviation in the measured normal xx pressure values.
- **'pyy_std'** (*float*) - The standard deviation in the measured normal yy pressure values.
- **'pzz_std'** (*float*) - The standard deviation in the measured normal zz pressure values.
- **'Epot_std'** (*float*) - The standard deviation in the measured total potential energy values.
- **'dx'** (*float*) - The computed diffusion constant along the x-direction.
- **'dy'** (*float*) - The computed diffusion constant along the y-direction.
- **'dz'** (*float*) - The computed diffusion constant along the y-direction.
- **'d'** (*float*) - The total computed diffusion constant.

__length2_pertime_unit__ is the units to display the computed diffusion constants in.

In [13]:
length2_pertime_unit = 'm^2/s'

In [14]:
print('Dx =', uc.get_in_units(results_dict['dx'], length2_pertime_unit), length2_pertime_unit)
print('Dy =', uc.get_in_units(results_dict['dy'], length2_pertime_unit), length2_pertime_unit)
print('Dz =', uc.get_in_units(results_dict['dz'], length2_pertime_unit), length2_pertime_unit)
print('D = ', uc.get_in_units(results_dict['d'], length2_pertime_unit), length2_pertime_unit)

Dx = -2.7193712398064232e-11 m^2/s
Dy = 4.744574795093042e-11 m^2/s
Dz = -2.725219690344457e-11 m^2/s
D =  -2.333387116857553e-12 m^2/s
